11785 - Midterm Submission

The following code is for training a self supervised model that predicts depth from images, where training is done using stereo pairs from the KITTI dataset

In [ ]:
import os
import random
import numpy as np
import copy
from PIL import Image  

import PIL.Image as pil
from tqdm import tqdm
import torch
from torch import autograd
import torch.utils.data as data
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torch.utils.model_zoo as model_zoo
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import skimage.transform
from collections import Counter
import time

import json


## Some helper functions are used from the original implementation
from utils import *
from kitti_utils import *
from layers import *


import datasets
from depthDecoder import *
# import networks
from IPython import embed


cuda = torch.cuda.is_available()
cuda
!nvidia-smi

In [ ]:
class Conv3x3(nn.Module):
    
    def __init__(self, in_channels, out_channels):
        super(Conv3x3, self).__init__()

        self.pad = nn.ReflectionPad2d(1)

        self.conv = nn.Conv2d(int(in_channels), int(out_channels), 3)

    def forward(self, x):
        out = self.pad(x)
        out = self.conv(out)
        return out


class ConvBlock(nn.Module):


    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()

        self.conv = Conv3x3(in_channels, out_channels)
        self.nonlin = nn.ELU(inplace=True)

    def forward(self, x):
        out = self.conv(x)
        out = self.nonlin(out)
        return out

class Decoder(nn.Module):
    def __init__(self, num_ch_enc, scales=range(4), num_output_channels=1, use_skips=True):
        super(Decoder, self).__init__()

        self.num_output_channels = num_output_channels
        self.use_skips = use_skips
        self.scales = scales

        self.num_ch_enc = num_ch_enc
        self.num_ch_dec = np.array([16, 32, 64, 128, 256])

        self.convs = OrderedDict()
        for i in range(4, -1, -1):
            # upconv_0
            num_ch_in = self.num_ch_enc[-1] if i == 4 else self.num_ch_dec[i + 1]
            num_ch_out = self.num_ch_dec[i]
            self.convs[("upconv", i, 0)] = ConvBlock(num_ch_in, num_ch_out)
        
            num_ch_in = self.num_ch_dec[i]
            if self.use_skips and i > 0:
                num_ch_in += self.num_ch_enc[i - 1]
            num_ch_out = self.num_ch_dec[i]
            self.convs[("upconv", i, 1)] = ConvBlock(num_ch_in, num_ch_out)

        for s in self.scales:
            self.convs[("dispconv", s)] = Conv3x3(self.num_ch_dec[s], self.num_output_channels)

        self.decoder = nn.ModuleList(list(self.convs.values()))
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_features):
        self.outputs = {}

        # decoder
        x = input_features[-1]
        for i in range(4, -1, -1):
            x = self.convs[("upconv", i, 0)](x)
            x = [F.interpolate(x, scale_factor=2, mode="nearest")]
            if self.use_skips and i > 0:
                x += [input_features[i - 1]]
            x = torch.cat(x, 1)
            x = self.convs[("upconv", i, 1)](x)
            if i in self.scales:
                self.outputs[("disp", i)] = self.sigmoid(self.convs[("dispconv", i)](x))

        return self.outputs


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, input_channel, channel, kernel_size, stride_val):
        super(ResBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(input_channel, channel, kernel_size, stride_val,padding=1,bias=False)
        self.bn1 = nn.BatchNorm2d(channel)
        self.conv2 = nn.Conv2d(channel, channel, kernel_size, stride=1, padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(channel)
        
        self.conv3_sc = torch.nn.Conv2d(input_channel,channel,kernel_size=1,stride=stride_val,bias = False)
        self.bn3_sc = torch.nn.BatchNorm2d(channel)
        
        self.conv3 = nn.Conv2d(channel, channel, kernel_size, stride=1,padding=1,bias=False)
        self.bn3 = nn.BatchNorm2d(channel)
        self.conv4 = nn.Conv2d(channel, channel, kernel_size, stride=1,padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(channel)
    
    def forward(self, x):
        
        identity = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        identity = self.bn3_sc(self.conv3_sc(identity))
        out += identity
        out = F.relu(out)
        
        identity1 = out
        out1 = F.relu(self.bn3(self.conv3(out)))
        out1 = self.bn4(self.conv3(out1))
        out1 += identity1
        out1 = F.relu(out1)
        return out     

class ResnetEncoder(nn.Module):
    def __init__(self):
        super(ResnetEncoder, self).__init__()
        
        self.num_ch_enc = np.array([64, 64, 128, 256, 512])

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = ResBlock(input_channel = 64, channel = 64, kernel_size = 3, stride_val = 1)
        self.layer2 = ResBlock(input_channel = 64, channel = 128, kernel_size = 3, stride_val = 2)
        self.layer3 = ResBlock(input_channel = 128, channel = 256, kernel_size = 3, stride_val = 2)
        self.layer4 = ResBlock(input_channel = 256, channel = 512, kernel_size = 3, stride_val = 2)

    
    
    def forward(self, input_image):
        self.features = []
        x = (input_image - 0.45) / 0.225
        x = self.conv1(x)
        x = self.bn1(x)
        self.features.append(self.relu(x))
        self.features.append(self.layer1(self.maxpool(self.features[-1])))
        self.features.append(self.layer2(self.features[-1]))
        self.features.append(self.layer3(self.features[-1]))
        self.features.append(self.layer4(self.features[-1]))

        return self.features

In [ ]:
def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

In [ ]:
class MainDataset(data.Dataset):
    
    def __init__(self,data_path,filenames,height,width,frame_idxs,num_scales,is_train=False,img_ext='.jpg'):
        super(MainDataset, self).__init__()
        
        self.data_path = data_path
        self.filenames = filenames
        self.height = height
        self.width = width
        self.num_scales = num_scales
        self.frame_idxs = frame_idxs
        self.is_train = is_train
        self.img_ext = img_ext
        
        self.interp = Image.ANTIALIAS
        self.loader = pil_loader
        self.to_tensor = transforms.ToTensor()
        
        try:
            self.brightness = (0.8, 1.2)
            self.contrast = (0.8, 1.2)
            self.saturation = (0.8, 1.2)
            self.hue = (-0.1, 0.1)
            transforms.ColorJitter.get_params(
                self.brightness, self.contrast, self.saturation, self.hue)
        except TypeError:
            self.brightness = 0.2
            self.contrast = 0.2
            self.saturation = 0.2
            self.hue = 0.1

               
        self.resize = {}
        for i in range(self.num_scales):
            s = 2 ** i
            self.resize[i] = transforms.Resize((self.height // s, self.width // s),interpolation=self.interp)
            
        self.load_depth = self.check_depth()
        
        self.K = np.array([[0.58, 0, 0.5, 0],
                           [0, 1.92, 0.5, 0],
                           [0, 0, 1, 0],
                           [0, 0, 0, 1]], dtype=np.float32)

        self.full_res_shape = (1242, 375)
        self.side_map = {"2": 2, "3": 3, "l": 2, "r": 3}
        
        
    def __len__(self):
        return len(self.filenames)
    
    def preprocess(self, inputs, color_aug):
        
        for k in list(inputs):
            frame = inputs[k]
            if "color" in k:
                n, im, i = k
                for i in range(self.num_scales):
                    inputs[(n, im, i)] = self.resize[i](inputs[(n, im, i - 1)])

        for k in list(inputs):
            f = inputs[k]
            if "color" in k:
                n, im, i = k
                inputs[(n, im, i)] = self.to_tensor(f)
                inputs[(n + "_aug", im, i)] = self.to_tensor(color_aug(f))
                
    

                    
    def check_depth(self):
        line = self.filenames[0].split()
        scene_name = line[0]
        frame_index = int(line[1])

        velo_filename = os.path.join(
            self.data_path,
            scene_name,
            "velodyne_points/data/{:010d}.bin".format(int(frame_index)))

        return os.path.isfile(velo_filename)
    
    def get_color(self, folder, frame_index, side, do_flip):
    #def get_color(self, folder, frame_index, side):
        
        color = self.loader(self.get_image_path(folder, frame_index, side))
        
        if do_flip:
            color = color.transpose(pil.FLIP_LEFT_RIGHT)

        return color
    
    def get_image_path(self, folder, frame_index, side):
        f_str = "{:010d}{}".format(frame_index, self.img_ext)
        image_path = os.path.join(
            self.data_path, folder, "image_0{}/data".format(self.side_map[side]), f_str)
        return image_path

    def get_depth(self, folder, frame_index, side):
        calib_path = os.path.join(self.data_path, folder.split("/")[0])

        velo_filename = os.path.join(
            self.data_path,
            folder,
            "velodyne_points/data/{:010d}.bin".format(int(frame_index)))

        depth_gt = generate_depth_map(calib_path, velo_filename, self.side_map[side])
        depth_gt = skimage.transform.resize(
            depth_gt, self.full_res_shape[::-1], order=0, preserve_range=True, mode='constant')


        return depth_gt
    

                    
    def __getitem__(self, index):
        
        inputs = {}
        
        do_color_aug = self.is_train and random.random() > 0.5
        do_flip = self.is_train and random.random() > 0.5
        
        line = self.filenames[index].split()
        folder = line[0]
        
        
        
        if len(line) == 3:
            frame_index = int(line[1])
            side = line[2]
        else:
            frame_index = 0
            side = None

        for i in self.frame_idxs:
            if i == "s":
                other_side = {"r": "l", "l": "r"}[side]
                inputs[("color", i, -1)] = self.get_color(folder, frame_index, other_side, do_flip)
                #inputs[("color", i, -1)] = self.get_color(folder, frame_index, other_side)
            else:
                inputs[("color", i, -1)] = self.get_color(folder, frame_index + i, side, do_flip)
                #inputs[("color", i, -1)] = self.get_color(folder, frame_index + i, side)
                
        for scale in range(self.num_scales):
            K = self.K.copy()

            K[0, :] *= self.width // (2 ** scale)
            K[1, :] *= self.height // (2 ** scale)

            inv_K = np.linalg.pinv(K)

            inputs[("K", scale)] = torch.from_numpy(K)
            inputs[("inv_K", scale)] = torch.from_numpy(inv_K)

        if do_color_aug:
            color_aug = transforms.ColorJitter.get_params(
                self.brightness, self.contrast, self.saturation, self.hue)
        else:
            color_aug =  (lambda x: x)
            
        self.preprocess(inputs,color_aug)
        
        for i in self.frame_idxs:
            del inputs[("color", i, -1)]
            del inputs[("color_aug", i, -1)]
            
        if self.load_depth:
            depth_gt = self.get_depth(folder, frame_index, side)
            inputs["depth_gt"] = np.expand_dims(depth_gt, 0)
            inputs["depth_gt"] = torch.from_numpy(inputs["depth_gt"].astype(np.float32))
            
        if "s" in self.frame_idxs:
            stereo_T = np.eye(4, dtype=np.float32)
            baseline_sign =  +1 #-1 if do_flip else 1
            side_sign = -1 if side == "l" else 1
            stereo_T[0, 3] = side_sign * baseline_sign * 0.1

            inputs["stereo_T"] = torch.from_numpy(stereo_T)

        return inputs

In [ ]:
## Load the data according to files listed in eigen_full split 

dataset = datasets.KITTIRAWDataset
data_path = "/home/ubuntu/monodepth2/kitti_data"
fpath_train = "/home/ubuntu/monodepth2/splits/eigen_full/train_files.txt"
f_train = open(fpath_train)
train_filenames = f_train.readlines()
fpath_val = "/home/ubuntu/monodepth2/splits/eigen_full/val_files.txt"
f_val = open(fpath_val)
val_filenames = f_val.readlines()

In [ ]:
## Size to rescale all input images to
height = 192
width = 640

## Scales of the output image from each layer of the decoder
## output scale = input scale/2**(n) for n in scales
scales = np.arange(4)

img_ext = '.jpg'
device = torch.device("cuda" if cuda else "cpu")

## List of frames that the dataloader must return 
## 0 - present frame
## 's' - stereo couple frame
frame_ids = [0,'s']



weights_pretrained = True

num_scales = len(scales)
num_input_frames = len(frame_ids)

learning_rate = 0.0001

In [ ]:
batch_size = 40
num_workers = 20

train_dataset = dataset(data_path, train_filenames, height, width,frame_ids,num_scales,is_train=True, img_ext=img_ext)
train_loader = DataLoader(train_dataset, batch_size, True,num_workers=num_workers, pin_memory=True, drop_last=True)

val_dataset = dataset(data_path, val_filenames, height, width,frame_ids, 4, is_train=False, img_ext=img_ext)
val_loader = DataLoader(val_dataset, batch_size, True,num_workers=num_workers, pin_memory=True, drop_last=True)

In [9]:
class DepthEncoderModule(nn.Module):
    def __init__(self, layers):
        super(DepthEncoderModule, self).__init__()
        self.num_ch_enc = np.array([64, 64, 128, 256, 512])
        pretrained = True
        self.encoder = models.resnet34(pretrained)

    def forward(self, input_image):
        self.features = []
        x = (input_image - 0.45) / 0.225
        x = self.encoder.conv1(x)
        x = self.encoder.bn1(x)
        self.features.append(self.encoder.relu(x))
        self.features.append(self.encoder.layer1(self.encoder.maxpool(self.features[-1])))
        self.features.append(self.encoder.layer2(self.features[-1]))
        self.features.append(self.encoder.layer3(self.features[-1]))
        self.features.append(self.encoder.layer4(self.features[-1]))

        return self.features

In [10]:
params = []

## Initialize the models

# encoder = ResnetEncoder()
encoder = DepthEncoderModule(layers = 34)
params += list(encoder.parameters())


decoder = Decoder(encoder.num_ch_enc, scales)
# decoder = Decoder(encoder.num_ch_enc, scales)
params += list(decoder.parameters())


ssim = SSIM()
ssim.to(device)

encoder.to(device)
decoder.to(device)
optimizer = optim.Adam(params, learning_rate)

lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 15, 0.1)

depth_metric_names = [
            "de/abs_rel", "de/sq_rel", "de/rms", "de/log_rms", "da/a1", "da/a2", "da/a3"]


In [11]:
backproject_depth = {}
project_3d = {}
for scale in scales:
    h = height // (2 ** scale)
    w = width // (2 ** scale)

    backproject_depth[scale] = BackprojectDepth(batch_size, h, w)
    backproject_depth[scale].to(device)

    project_3d[scale] = Project3D(batch_size, h, w)
    project_3d[scale].to(device)
#print("There are {:d} training items and {:d} validation items\n".format(len(train_dataset), len(val_dataset)))

In [12]:

def generate_images_pred(inputs, outputs):
        
   
    source_scale = 0
    
    for scale in scales:
        disp = outputs[("disp", scale)]

        disp = F.interpolate(disp, [height, width], mode="bilinear",align_corners=False)
        
        depth = 1 / (0.01 + 9.99*disp)
 
        outputs[("depth", 0, scale)] = depth

        T = inputs["stereo_T"]

        cam_points = backproject_depth[source_scale](depth, inputs[("inv_K", source_scale)])
        pix_coords = project_3d[source_scale](cam_points, inputs[("K", source_scale)], T)
        outputs[("color", 's', scale)] = F.grid_sample(inputs[("color", 's', source_scale)],
                pix_coords,
                padding_mode="border",align_corners=False)

    return outputs   


In [13]:
import torch
from math import exp
import torch.nn.functional as F

def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)*2/float(2*sigma*2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel=1):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window

def ssim_new(img1, img2, val_range, window_size=11, window=None, size_average=True, full=False):
    L = val_range

    padd = 0
    (_, channel, height, width) = img1.size()
    if window is None:
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channel).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=padd, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padd, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=padd, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padd, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padd, groups=channel) - mu1_mu2

    C1 = (0.01 * L) ** 2
    C2 = (0.03 * L) ** 2

    v1 = 2.0 * sigma12 + C2
    v2 = sigma1_sq + sigma2_sq + C2
    cs = torch.mean(v1 / v2)  # contrast sensitivity

    ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)

    if size_average:
        ret = ssim_map.mean()
    else:
        ret = ssim_map.mean(1).mean(1).mean(1)

    if full:
        return ret, cs

    return ret

## Compute Losses

In [14]:

def compute_reprojection_loss(pred, target):

        l1_loss = (torch.abs(target - pred)).mean(1,True)
#         l2_loss = torch.sqrt(((target - pred)**2).mean(1, True))
        
        #ssim_loss = ssim(pred, target).mean(1, True)
        ssim_loss = torch.clamp((1 - ssim_new(pred, target,1)) * 0.5, 0, 1)
        reprojection_loss = 0.85*ssim_loss + 0.15 * l1_loss
#         print('reprojection ', reprojection_loss)
        return reprojection_loss


In [15]:
def get_smooth_loss(disp, img):

    grad_disp_x = torch.abs(disp[:, :, :, :-1] - disp[:, :, :, 1:])
    grad_disp_y = torch.abs(disp[:, :, :-1, :] - disp[:, :, 1:, :])

    grad_img_x = torch.mean(torch.abs(img[:, :, :, :-1] - img[:, :, :, 1:]), 1, keepdim=True)
    grad_img_y = torch.mean(torch.abs(img[:, :, :-1, :] - img[:, :, 1:, :]), 1, keepdim=True)

    grad_disp_x *= torch.exp(-grad_img_x)
    grad_disp_y *= torch.exp(-grad_img_y)

    return grad_disp_x.mean() + grad_disp_y.mean()


In [16]:

def compute_losses(inputs, outputs):

        losses = {}
        total_loss = 0
        

        for scale in scales:
            loss = 0
            reprojection_losses = []

            source_scale = 0

            disp = outputs[("disp", scale)]
            color = inputs[("color", 0, scale)]
            target = inputs[("color", 0, source_scale)]


            pred = outputs[("color", 's', scale)]
            reprojection_losses.append(compute_reprojection_loss(pred, target))

            reprojection_losses = torch.cat(reprojection_losses, 1)
            to_optimise, idxs = torch.min(reprojection_losses, dim=1)
            #to_optimise = reprojection_losses
            loss = loss + to_optimise.mean()

            norm_disp = disp / (disp.mean(2, True).mean(3, True)+ 1e-7)
            smooth_loss = get_smooth_loss(norm_disp, color)

            loss =  loss + ((1e-3)*smooth_loss) / (2 ** scale)
            total_loss = total_loss + loss
            losses["loss/{}".format(scale)] = loss

        total_loss = total_loss/num_scales
        losses["loss"] = total_loss
        return losses



In [17]:

def compute_depth_losses(inputs, outputs, losses):


        depth_pred = outputs[("depth", 0, 0)]
        depth_pred = torch.clamp(F.interpolate(
            depth_pred, [375, 1242], mode="bilinear", align_corners=False), min=1e-3, max=80)
        depth_pred = depth_pred.detach()

        depth_gt = inputs["depth_gt"]
        mask = depth_gt > 0

        crop_mask = torch.zeros_like(mask)
        crop_mask[:, :, 153:371, 44:1197] = 1
        mask = mask * crop_mask

        gt = depth_gt[mask]
        pred = depth_pred[mask]
        pred = pred*(torch.median(gt) / torch.median(pred))

        pred = torch.clamp(pred, min=1e-3, max=80)
        thresh = torch.max((gt / pred), (pred / gt))
        a1 = (thresh < 1.25     ).float().mean()
        a2 = (thresh < 1.25 ** 2).float().mean()
        a3 = (thresh < 1.25 ** 3).float().mean()

        rmse = torch.sqrt(((gt - pred)**2).mean())
        rmse_log = torch.sqrt(((torch.log(gt + 1e-6) - torch.log(pred + 1e-6))**2).mean())
        abs_rel = torch.mean(torch.abs(gt - pred) / gt)
        sq_rel = torch.mean(((gt - pred)**2 )/ gt)
        
        return np.array([abs_rel.item(),sq_rel.item(),rmse.item(),rmse_log.item(),a1.item(),a2.item(),a3.item()])


In [18]:
def train(encoder,decoder,train_loader,val_loader,num_epochs):
              
        encoder.train()
        decoder.train()
        
        for epoch in range(0,num_epochs):
            
            total_loss = 0 
            outer = tqdm(total=(len(train_loader)), desc='Training Epoch', position=0)

            for batch_idx, inputs in enumerate(train_loader):

                outer.update(1)
                optimizer.zero_grad()


                for key, ipt in inputs.items():
                    inputs[key] = ipt.to(device)

                features = encoder(inputs["color_aug", 0, 0])
                outputsl = decoder(features)

       
                outputs = generate_images_pred(inputs, outputsl)
                

                losses = compute_losses(inputs, outputs)
                total_loss = total_loss + losses['loss'].item()
                losses["loss"].backward()
                optimizer.step()
                lr_scheduler.step()

                del inputs, outputs, features
                if batch_idx%500==0 and batch_idx>0:
                    print('Batch No: ',batch_idx)
                    #print('Loss: ',total_loss/(batch_idx+1))
                    print('Loss: ',losses["loss"].cpu().data)
                del losses
                    
  
            total_loss = total_loss/len(train_loader)
            print('Train Loss at Epoch_{}:'.format(epoch+1),total_loss)
            validation(encoder,decoder,val_loader)
            run_images(epoch)

In [19]:
def validation(encoder,decoder,val_loader):
    
    with torch.no_grad():
        encoder.eval()
        decoder.eval()
        total_loss = 0
        metrics = np.zeros((7))
        for batch_idx, inputs in enumerate(val_loader):

            before_op_time = time.time()

            for key, ipt in inputs.items():
                inputs[key] = ipt.to(device)

            features = encoder(inputs["color_aug", 0, 0])
            outputs = decoder(features)
            
            outputs = generate_images_pred(inputs, outputs)
            losses = compute_losses(inputs, outputs)
            total_loss = total_loss + losses['loss'].item()
         
            metrics = metrics + compute_depth_losses(inputs, outputs, losses)
            del inputs, outputs, losses,features
         
        total_loss = total_loss/len(val_loader)
        print('Total Loss (Validation): ',total_loss)
        print('Error Metrics: ',metrics/len(val_loader))      

In [ ]:
num_epochs = 7
train(encoder,decoder,train_loader,val_loader,num_epochs)

Training Epoch:  44%|████▍     | 502/1130 [11:42<35:26,  3.39s/it]  

Batch No:  500
Loss:  tensor(0.3007)


Training Epoch:  89%|████████▊ | 1002/1130 [22:39<06:52,  3.23s/it]

Batch No:  1000
Loss:  tensor(0.3059)


Training Epoch: 100%|██████████| 1130/1130 [25:04<00:00,  1.90it/s]

Train Loss at Epoch_1: 0.30285951938249367
Total Loss (Validation):  0.30100185153159226
Error Metrics:  [ 0.48467822  5.08808824 12.44722319  0.62208502  0.27230702  0.52409697
  0.75657267]


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type DepthEncoderModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ConvBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Conv3x3. I

In [ ]:

num_epochs = 1
train(encoder,decoder,train_loader,val_loader,num_epochs)

In [ ]:

num_epochs = 1
train(encoder,decoder,train_loader,val_loader,num_epochs)

In [ ]:

num_epochs = 1
train(encoder,decoder,train_loader,val_loader,num_epochs)

In [ ]:
run_images(4)

In [ ]:

num_epochs = 7
train(encoder,decoder,train_loader,val_loader,num_epochs)

In [ ]:
num_epochs = 10
train(encoder,decoder,train_loader,val_loader,num_epochs)

In [20]:
folder_name = 'ModifiedL2Loss_Stereo'

In [21]:
def saveModel(epoch_num):
    torch.save(encoder,'StereoOutput/Models/Resnet18/'+folder_name+'/DepthEncoder_'+str(epoch_num)+'.pt')
    torch.save(decoder,'StereoOutput/Models/Resnet18/'+folder_name+'/DepthDecoder_'+str(epoch_num)+'.pt')
    #torch.save(pose_enc,'StereoOutput/Models/Resnet18/'+folder_name+'/PoseEncoder_'+str(epoch_num)+'.pt')
    #torch.save(pose_dec,'StereoOutput/Models/Resnet18/'+folder_name+'/PoseDecoder_'+str(epoch_num)+'.pt')

In [22]:
def run_images(epoch_num):
    directory = 'Iteration '+ str(epoch_num)+'/'
    parent_dir = 'StereoOutput/Resnet18/'+folder_name+'/'
    path = os.path.join(parent_dir, directory) 
    os.mkdir(path) 
    image_output_path = parent_dir+directory
    for i in range(1, 9):
        image_name = 'Test'+str(i)+'.jpg'
        image_path = '/home/ubuntu/monodepth2/assets/'+image_name
        input_image = pil.open(image_path).convert('RGB')
        original_width, original_height = input_image.size
        input_image_resized = input_image.resize((width, height), pil.LANCZOS)
        input_image_pytorch = transforms.ToTensor()(input_image_resized).unsqueeze(0)

        with torch.no_grad():
            encoder.eval()
            decoder.eval()
            features = encoder(input_image_pytorch.to(device))
            outputs = decoder(features)

        disp = outputs[("disp", 0)]
        disp_resized = torch.nn.functional.interpolate(disp,(original_height, original_width), mode="bilinear", align_corners=True)

        # Saving colormapped depth image
        disp_resized_np = disp_resized.squeeze().cpu().numpy()
        vmax = np.percentile(disp_resized_np, 95)

        plt.figure(figsize=(10, 10))
        plt.subplot(211)
        plt.imshow(input_image)
        plt.title("Input", fontsize=22)
        plt.axis('off')

        plt.subplot(212)
        plt.imshow(disp_resized_np, cmap='magma', vmax=vmax)
        plt.title("Disparity prediction", fontsize=22)
        plt.axis('off')
        plt.savefig(image_output_path+image_name,dpi = 100)
    saveModel(epoch_num)

In [ ]:
run_images(1)

In [ ]:
i = 0
total_loss = 0
train_loader = val_loader
for batch_idx, inputs in enumerate(train_loader):
                optimizer.zero_grad()

                for key, ipt in inputs.items():
                    inputs[key] = ipt.to(device)

                features = encoder(inputs["color_aug", 0, 0])
                outputs = decoder(features)
               
                o1 = generate_images_pred(inputs, outputs)
                
                losses = compute_losses(inputs, o1)
                
                total_loss = total_loss + losses['loss'].item()
                optimizer.zero_grad()
                losses["loss"].backward()
                optimizer.step()
                i = i+1
                
                lr_scheduler.step()
                
                if i>2:
                    break
                   
                del inputs, outputs, losses,features,o1
                

In [ ]:
outputs[('disp', 3)]

In [ ]:
image_path = "/home/ubuntu/monodepth2/assets/Home_Hero5.jpg"
input_image = pil.open(image_path).convert('RGB')
original_width, original_height = input_image.size
input_image_resized = input_image.resize((width, height), pil.LANCZOS)
input_image_pytorch = transforms.ToTensor()(input_image_resized).unsqueeze(0)

In [ ]:
with torch.no_grad():
    encoder.eval()
    decoder.eval()
    features = encoder(input_image_pytorch.to(device))
    outputs = decoder(features)

disp = outputs[("disp", 0)]

In [ ]:
disp_resized = torch.nn.functional.interpolate(disp,
    (original_height, original_width), mode="bilinear", align_corners=True)

# Saving colormapped depth image
disp_resized_np = disp_resized.squeeze().cpu().numpy()
vmax = np.percentile(disp_resized_np, 95)

plt.figure(figsize=(10, 10))
plt.subplot(211)
plt.imshow(input_image)
plt.title("Input", fontsize=22)
plt.axis('off')

plt.subplot(212)
plt.imshow(disp_resized_np, cmap='magma', vmax=vmax)
plt.title("Disparity prediction", fontsize=22)
plt.axis('off');
# plt.save()

In [ ]:
a = val_dataset[0][('color_aug', 's', 0)]
b = val_dataset[0][('color', 0, 0)]

In [ ]:
class ss(nn.Module):
    """Layer to compute the SSIM loss between a pair of images
    """
    def __init__(self):
        super(ss, self).__init__()
        self.mu_x_pool   = nn.AvgPool2d(3, 1)
        self.mu_y_pool   = nn.AvgPool2d(3, 1)
        self.sig_x_pool  = nn.AvgPool2d(3, 1)
        self.sig_y_pool  = nn.AvgPool2d(3, 1)
        self.sig_xy_pool = nn.AvgPool2d(3, 1)

        self.refl = nn.ReflectionPad2d(1)

        self.C1 = 0.01 ** 2
        self.C2 = 0.03 ** 2

    def forward(self, x, y):
        x = self.refl(x)
        y = self.refl(y)

        mu_x = self.mu_x_pool(x)
        mu_y = self.mu_y_pool(y)

        sigma_x  = self.sig_x_pool(x ** 2) - mu_x ** 2
        sigma_y  = self.sig_y_pool(y ** 2) - mu_y ** 2
        sigma_xy = self.sig_xy_pool(x * y) - mu_x * mu_y

        SSIM_n = (2 * mu_x * mu_y + self.C1) * (2 * sigma_xy + self.C2)
        SSIM_d = (mu_x ** 2 + mu_y ** 2 + self.C1) * (sigma_x + sigma_y + self.C2)

        return torch.clamp((1 - SSIM_n / SSIM_d) / 2, 0, 1)


In [ ]:
s = ss()

In [ ]:
validation(encoder,decoder,val_loader)

In [ ]:
import os

test_path = "/home/ubuntu/monodepth2/kitti_data"

ftest = "/home/ubuntu/monodepth2/splits/eigen/test_files.txt"
test_filenames = open(ftest).readlines()


for f in test_filenames:
    path = f.split(" ")
    
    
    if path[2]=='l\n':
        image_path = os.path.join(test_path,path[0],"image_03/data",path[1])
    else:
        image_path = os.path.join(test_path,path[0],"image_04/data",path[1])
    print(image_path)
    input_image = pil.open(image_path+".jpg").convert('RGB')
    original_width, original_height = input_image.size
    input_image_resized = input_image.resize((width, height), pil.LANCZOS)
    input_image_pytorch = transforms.ToTensor()(input_image_resized).unsqueeze(0)
    
    with torch.no_grad():
        encoder.eval()
        decoder.eval()
        features = encoder(input_image_pytorch.to(device))
        outputs = decoder(features)

    disp = outputs[("disp", 0)]
    disp_resized = torch.nn.functional.interpolate(disp,
    (original_height, original_width), mode="bilinear", align_corners=True)

    # Saving colormapped depth image
    disp_resized_np = disp_resized.squeeze().cpu().numpy()
    vmax = np.percentile(disp_resized_np, 95)

    plt.figure(figsize=(10, 10))
    plt.subplot(211)
    plt.imshow(input_image)
    plt.title("Input", fontsize=22)
    plt.axis('off')

    plt.subplot(212)
    plt.imshow(disp_resized_np, cmap='magma', vmax=vmax)
    plt.title("Disparity prediction", fontsize=22)
    plt.axis('off');
    
    plt.savefig('output_imgs_10epochs/'+path[1]+'.png',dpi = 100)
    
    plt.close('all')
    del outputs,features



In [ ]:
image_path = "/home/ubuntu/monodepth2/assets/test_image.jpg"
input_image = pil.open(image_path).convert('RGB')
original_width, original_height = input_image.size
input_image_resized = input_image.resize((width, height), pil.LANCZOS)
input_image_pytorch = transforms.ToTensor()(input_image_resized).unsqueeze(0)

In [ ]:
torch.save(encoder,'encoder10e.pt')
torch.save(decoder,'decoder10e.pt')

In [ ]:
encoder = torch.load('encoder10e.pt')
decoder = torch.load('decoder10e.pt')

In [ ]:
print(encoder)